# Contour-following MPC

#### Presented by: Alejandro Astudillo and Ajay Sathya

An implementation of a contour-following MPC using **Tasho** - A model predictive control toolchain for constraint-based task specification of robot motions

### Optimization problem
$
\begin{aligned}
minimize\ & \int_{0}^{t_f} \left\Vert\begin{bmatrix}\dot{s} - \dot{s}_{ref}(s)\\ e_{T}(q,s) \\ x \\ u \\ \nu \end{bmatrix}\right\Vert^{2}_{\mathbf{W}} + \alpha_{l}l\ dt\\\\
s.t.\ \ \ \ & \dot{x} = f_{x}(x, u),\\
& \dot{\zeta}  = f_\zeta(\zeta,\nu),\\
& \underline{r} \leq r(x,u) \leq \overline{r},\\
& \underline{q} \leq q \leq \overline{q},\\
& ||e_{T}(q,s)||^{2} \leq \rho^{2} + l,\\
& l \geq 0,\\
& 0 \leq s \leq 1,\\
& x(0) = \hat{x}_{0},\\
& (x(t_{f}),\zeta(t_{f})) \in \varepsilon.
\end{aligned}
$  


$q(t)$ is the joint angles vector, $s$ is the path-progress variable, $f_{x}(x(t), u(t))$ are the robot dynamics, $l(t)$ are slack variables to avoid infeasibilities in tunnel constraints, $\hat{x}_{0}$ is the initial state at each MPC evaluation (may be estimated), $\varepsilon$ is the set of desired final states for the robot and path dynamics, $e_{T}(q,s)$ represents the position and orientation error of the end-effector with respect to a path reference, and $\rho$ defines the maximum allowed deviation of the end-effector from such reference.

On the other hand, $r(x(t),u(t))$ may be a box constraint if the torques $r$ are set to be the inputs $(u(t) = r(x(t),u(t)))$, or may be a nonlinear function corresponding the the inverse dynamics of the robot if the joint accelerations are the inputs of the system $(u(t) = \ddot{q}(t))$.
  
  
First, we need to import the relevant modules from Tasho and other dependencies (casadi, numpy)

In [1]:
from tasho import task_prototype_rockit as tp
from tasho import input_resolution, world_simulator
from tasho import robot as rob
from tasho import MPC
from tasho.utils import geometry
import casadi as cs
import numpy as np

### Define the robot to be used within the task context

Import the robot object from the robot's repository (includes functions for FD, ID, FK, joint limits, etc)

In [2]:
robot_choice = "yumi"
ocp_control = "acceleration_resolved"  #'torque_resolved'

robot = rob.Robot(robot_choice, analytical_derivatives=True)

Loading robot params from json


Update robot's parameters if needed

In [3]:
if ocp_control == "acceleration_resolved":
    max_joint_acc = 240 * cs.pi / 180
    robot.set_joint_acceleration_limits(lb=-max_joint_acc, ub=max_joint_acc)

Define initial conditions of the robot

In [4]:
left_arm_q_init = [-1.35, -8.72e-01, 2.18, 6.78e-01, 2.08, -9.76e-01, -1.71, 1.65e-03, 1.65e-03]
# Right arm in home configuration
right_arm_q_init = [0, -2.26, -2.35, 0.52, 0.025, 0.749, 0, 0, 0]

q_init = np.array(left_arm_q_init + right_arm_q_init).T

q_dot_init = [0] * robot.ndof

## Task spacification - Contour following

Select prediction horizon and sample time for the MPC execution

In [5]:
horizon_size = 16
t_mpc = 0.01

Initialize the task context object

In [6]:
tc = tp.task_context(horizon_size * t_mpc, horizon_steps=horizon_size)

Define the input type of the robot (torque or acceleration) and, consequently, the robot dynamics $f_x(x,u)$ to be used within the task context

In [7]:
if ocp_control == "acceleration_resolved":
    q, q_dot, q_ddot, q0, q_dot0 = input_resolution.acceleration_resolved(tc, robot, {})
elif ocp_control == "torque_resolved":
    q, q_dot, q_ddot, tau, q0, q_dot0 = input_resolution.torque_resolved(
        tc, robot, {"forward_dynamics_constraints": False}
    )

Define path dynamics $f_\zeta(\zeta,\nu)$ based on path-progress variable $s$

$
\begin{aligned}
f_\zeta(\zeta,\nu) & = \begin{bmatrix}0 & 1 \\ 0 & 0 \end{bmatrix}\zeta + \begin{bmatrix}0 \\ 1 \end{bmatrix}\nu,\\
\zeta & = \begin{bmatrix}s \\ \dot{s}\end{bmatrix},\ \ \ \ \ \ \ \nu = \ddot{s},
\end{aligned}
$


In [8]:
s = tc.create_expression("s", "state", (1, 1))
s_dot = tc.create_expression("s_dot", "state", (1, 1))
s_ddot = tc.create_expression("s_ddot", "control", (1, 1))

tc.set_dynamics(s, s_dot)
tc.set_dynamics(s_dot, s_ddot)

Set $s(0)$ and $\dot{s}(0)$ as parameters

In [9]:
s0 = tc.create_expression("s0", "parameter", (1, 1))
s_dot0 = tc.create_expression("s_dot0", "parameter", (1, 1))

s_init_con = {"expression": s, "reference": s0}
s_dot_init_con = {"expression": s_dot, "reference": s_dot0}
init_constraints = {"initial_constraints": [s_init_con, s_dot_init_con]}
tc.add_task_constraint(init_constraints)

Add constraints for path-progress variable:   
$0 <= s <= 1$  
$\dot{s} >= 0$

In [10]:
s_con = {
    "hard": True,
    "lub": True,
    "expression": s,
    "lower_limits": 0,
    "upper_limits": 1,
}
s_dotcon = {
    "hard": True,
    "inequality": True,
    "expression": -s_dot,
    "upper_limits": 0,
}
s_path_constraints = {"path_constraints": [s_con, s_dotcon]}
tc.add_task_constraint(s_path_constraints)

### Define the contour to be followed

In [11]:
def contour_path(s):
    ee_fk_init = robot.fk(q_init)[7]
    ee_pos_init = ee_fk_init[:3, 3]
    ee_rot_init = ee_fk_init[:3, :3]

    sdotref = 0.1
    sdot_path = sdotref * (
        5.777783e-13 * s ** 5
        - 34.6153846154 * s ** 4
        + 69.2307692308 * s ** 3
        - 46.7307692308 * s ** 2
        + 12.1153846154 * s
        + 0.0515384615
    )

    a_p = 0.15
    z_p = 0.05
    pos_path = ee_pos_init + cs.vertcat(
        a_p * cs.sin(s * (2 * cs.pi)),
        0,
        a_p * cs.sin(s * (2 * cs.pi)) * cs.cos(s * (2 * cs.pi)),
    )
    rot_path = ee_rot_init
    # rot_path = np.array([[0, 1, 0], [1, 0, 0], [0, 0, 1]])

    return pos_path, rot_path, sdot_path

# Define contour/path based on the path-progress variable s
pos_path, rot_path, sdot_path = contour_path(s)

### Define end-effector position and orientation error  

$
\begin{aligned}
e_{T}(q,s) & = \begin{bmatrix} p_{ee}(q) - p_{ref}(s) \\ \frac{1}{2}(n_{ref}(s)\times n_{ee}(q) + s_{ref}(s)\times s_{ee}(q) + a_{ref}(s)\times a_{ee}(q)) \end{bmatrix},
\end{aligned}
$   
with $R_{ee} = \left[\begin{matrix} n_{ee} & s_{ee} & a_{ee} \end{matrix}\right]$ being the rotation matrix of the end-effector with respect to the robot joint angles $q$, and $R_{ref} = \left[\begin{matrix} n_{ref} & s_{ref} & a_{ref} \end{matrix}\right]$ the reference rotation matrix w.r.t the path-progress variable $s$  

In [12]:
def pos_err(q, s):
    ee_fk = robot.fk(q)[7]
    return ee_fk[:3, 3] - pos_path


def rot_err(q, s):
    ee_fk = robot.fk(q)[7]
    ee_rot_n = ee_fk[:3, 0]
    ee_rot_s = ee_fk[:3, 1]
    ee_rot_a = ee_fk[:3, 2]

    path_rot_n = rot_path[:, 0]
    path_rot_s = rot_path[:, 1]
    path_rot_a = rot_path[:, 2]

    return 0.5 * (
        geometry.cross_vec2vec(ee_rot_n, path_rot_n)
        + geometry.cross_vec2vec(ee_rot_s, path_rot_s)
        + geometry.cross_vec2vec(ee_rot_a, path_rot_a)
    )


### Add the tunnel constraint to the task context 
This constraint is managed as a soft constraint (adding a slack variable $l$ to the OCP)  
$\left\Vert\begin{bmatrix} p_{ee}(q) - p_{ref}(s)\\  \frac{1}{2}\begin{bmatrix}n_{ref}(s)\times n_{ee}(q) + s_{ref}(s)\times s_{ee}(q) + a_{ref}(s)\times a_{ee}(q)\end{bmatrix} \end{bmatrix}\right\Vert^{2}  \leq \rho^{2} - l$

In [13]:
tun_tunnel_con = { 
    "hard": False,
    "inequality": True,
    "expression": cs.vertcat(pos_err(q, s), rot_err(q, s)),
    "upper_limits": 0.01 ** 2,
    "gain": 100,
    "norm": "squaredL2",
}
tunnel_constraints = {"path_constraints": [tun_tunnel_con]}
tc.add_task_constraint(tunnel_constraints)

### Define objective

Add regularization terms to the objective

In [14]:
tc.add_regularization(expression=(s_dot - sdot_path), weight=20, norm="L2")
tc.add_regularization(expression=pos_err(q, s), weight=1e-1, norm="L2")
tc.add_regularization(expression=rot_err(q, s), weight=1e-1, norm="L2")

if ocp_control == "torque_resolved":
    tc.add_regularization(
        expression=tau, weight=4e-5, norm="L2", variable_type="control", reference=0
    )
if ocp_control == "acceleration_resolved":
    tc.add_regularization(
        expression=q_ddot,
        weight=1e-3,
        norm="L2",
        variable_type="control",
        reference=0,
    )
tc.add_regularization(
    expression=s_ddot, weight=4e-5, norm="L2", variable_type="control", reference=0
)

tc.add_regularization(
    expression=q[0:8], weight=1e-2, norm="L2", variable_type="state", reference=0
)
tc.add_regularization(
    expression=q_dot[0:8], weight=1e-2, norm="L2", variable_type="state", reference=0
)

Add terms at $t_f$ to the objective

In [15]:
tc.add_objective(
    tc.ocp.at_tf(
        1e-5
        * cs.sumsqr(
            cs.vertcat(
                1e-2 * q[0:8],
                10 * q_dot[0:8],
                1e-2 * (1 - s),
                10 * s_dot,
                10 * pos_err(q, s),
                10 * rot_err(q, s),
            )
        )
    )
)

### Set solver and discretization options

In [16]:
tc.set_ocp_solver("ipopt")

disc_settings = {
    "discretization method": "multiple shooting",
    "horizon size": horizon_size,
    "order": 1,
    "integration": "rk",
}
tc.set_discretization_settings(disc_settings)

### Set parameter values
In the defined task context, the parameters correspond to the initial state of the robot.

In [17]:
tc.ocp.set_value(q0, q_init)
tc.ocp.set_value(q_dot0, [0] * robot.ndof)
tc.ocp.set_value(s0, 0)
tc.ocp.set_value(s_dot0, 0)

### Solve the OCP that describes the task

In [18]:
sol = tc.solve_ocp()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2166
Number of nonzeros in inequality constraint Jacobian.:     1066
Number of nonzeros in Lagrangian Hessian.............:     1085

Total number of variables............................:      966
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

## MPC Simulation

Create world simulator based on pybullet

In [19]:
from tasho import world_simulator
import pybullet as p

obj = world_simulator.world_simulator(bullet_gui=False)

Add robot to the world environment

In [20]:
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
yumiID = obj.add_robot(position, orientation, "yumi")

Determine number of samples that the simulation should be executed

In [21]:
no_samples = int(t_mpc / obj.physics_ts)

Correspondence between joint numbers in bullet and OCP

In [22]:
joint_indices = [11, 12, 13, 14, 15, 16, 17, 18, 19, 1, 2, 3, 4, 5, 6, 7, 8, 9]

Begin the visualization by applying the initial control signal

In [23]:
obj.resetJointState(yumiID, joint_indices, q_init)
obj.setController(yumiID, "velocity", joint_indices, targetVelocities=q_dot_init)

### Define MPC parameters

In [24]:
mpc_params = {"world": obj}

In [25]:
q0_params_info = {
    "type": "joint_position",
    "joint_indices": joint_indices,
    "robotID": yumiID,
}
q_dot0_params_info = {
    "type": "joint_velocity",
    "joint_indices": joint_indices,
    "robotID": yumiID,
}
s0_params_info = {"type": "progress_variable", "state": True}
s_dot0_params_info = {"type": "progress_variable", "state": True}

mpc_params["params"] = {
    "q0": q0_params_info,
    "q_dot0": q_dot0_params_info,
    "s0": s0_params_info,
    "s_dot0": s_dot0_params_info,
    "robots": {yumiID: robot},
}

Replace the solver used in the task context (IPOPT) with the SQP method and QRQP for MPC execution

In [26]:
mpc_params["disc_settings"] = disc_settings
mpc_params["solver_name"] = "sqpmethod"
mpc_params["solver_params"] = {"qrqp": True}
mpc_params["t_mpc"] = t_mpc
mpc_params["control_type"] = "joint_velocity"  #'joint_torque'
mpc_params["control_info"] = {
    "robotID": yumiID,
    "discretization": "constant_acceleration",
    "joint_indices": joint_indices,
    "no_samples": no_samples,
}

### Define code-generation options
This options may include just-in-time compilation, compilation flags, selection of desired compiler, etc.

In [27]:
mpc_params["codegen"] = {
    "codegen": False,
    "filename": "mpc_c",
    "compilation": False,
    "compiler": "gcc",
    "flags": "-O3 -ffast-math -flto -funroll-loops -march=native -mfpmath=both -mvzeroupper",
    "use_external": False,
    "jit": False,
}

### Create monitor
Add monitor to check some termination criteria based on the path-progress variable $s$

In [28]:
tc.add_monitor(
    {
        "name": "termination_criteria",
        "expression": s,
        "reference": 0.99,
        "greater": True,
        "initial": True,
    }
)

### MPC object
Initialize MPC object based on the task context created previously and the defined MPC parameters

In [29]:
sim_type = "bullet_notrealtime"
mpc_obj = MPC.MPC(tc, sim_type, mpc_params)
mpc_obj.max_mpc_iter = 4000

Run the OCP with IPOPT once to get a good initial guess for the MPC, and code-generate the MPC function if required

In [30]:
mpc_obj.configMPC_fromcurrent()

None
This is Ipopt version 3.12.3, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:     2166
Number of nonzeros in inequality constraint Jacobian.:     1066
Number of nonzeros in Lagrangian Hessian.............:     1085

Total number of variables............................:      966
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      646
Total number of inequality constraints...............:      930
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:      880
        inequality constraints with only upper bounds:       50

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.9277794e-02 2.35e+00 6.01e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00   0

### Execute the MPC loop

In [31]:
mpc_obj.runMPC()

None
Before printing system dynamics
{'q0': [-1.35, -0.872, 2.18, 0.678, 2.08, -0.976, -1.71, 0.0, 0.00165, 0.0, -2.26, -2.35, 0.52, 0.025, 0.749, 0.0, 0.0, 0.0], 'q_dot0': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 's0': 0, 's_dot0': 0}
q
q_dot
s
s_dot
iter      objective    inf_pr    inf_du     ||d||  lg(rg) ls
   0   2.559161e-02  1.65e-03  1.00e+00  0.00e+00       -  0
   1   2.558883e-02  4.44e-16  3.56e-07  2.73e-01       -  1
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
          QP  | 119.41ms (119.41ms) 119.02ms (119.02ms)         1
  linesearch  | 153.00us (153.00us) 153.43us (153.43us)         1
      nlp_fg  | 115.00us (115.00us) 115.08us (115.08us)         1
    nlp_grad  | 369.00us (369.00us) 321.38us (321.38us)         1
  nlp_hess_l  |   1.29ms (  1.29ms)   1.18ms (  1.18ms)         1
  nlp_jac_fg  | 602.00us (301.00us) 519.77us (259.89us)         2
       total  | 122.97ms (122.97ms) 122.01ms (122.01

Before printing system dynamics
{'q0': [-1.3494603237450242, -0.8716562100974508, 2.179122162351522, 0.6777398885377939, 2.0791627535779016, -0.9756017065237534, -1.7093231504473672, 0.0, 0.0016758533825904655, 1.5724215341795384e-05, -2.2600692325048404, -2.349999939042566, 0.5199905318815613, 0.024781407616081035, 0.7493027033866735, 0.00024824412032223634, 0.0, 0.00015780572629291815], 'q_dot0': [0.009293020502434121, 0.005927095138773583, -0.015166520191354227, -0.004447503861118612, -0.014466983265263932, 0.006856294109355364, 0.011666301440334224, 0.0, 0.00022696411135864633, 0.00023586829927055297, -0.001039215946460656, 6.179925338412051e-07, -0.00014186205378635686, -0.0032788333596462417, 0.004539964601836283, 0.003723712067882001, 0.0, 0.001456509526009138], 's0': array([0.00075918]), 's_dot0': array([0.00606931])}
q
q_dot
s
s_dot
iter      objective    inf_pr    inf_du     ||d||  lg(rg) ls
   0   2.554375e-02  2.16e-04  7.30e-04  0.00e+00       -  0
      solver  :   t_proc

   0   2.548921e-02  1.58e-03  8.33e-04  0.00e+00       -  0
   1   2.549092e-02  7.25e-05  5.68e-07  1.73e-01       -  1
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
          QP  |   5.96ms (  5.96ms)   5.95ms (  5.95ms)         1
  linesearch  | 143.00us (143.00us) 143.20us (143.20us)         1
      nlp_fg  | 109.00us (109.00us) 109.22us (109.22us)         1
    nlp_grad  | 500.00us (500.00us) 459.65us (459.65us)         1
  nlp_hess_l  |   1.46ms (  1.46ms)   1.17ms (  1.17ms)         1
  nlp_jac_fg  | 508.00us (254.00us) 508.82us (254.41us)         2
       total  |   9.50ms (  9.50ms)   9.15ms (  9.15ms)         1
primal residual is : [[7.24957002e-05]]
0
None
Before printing system dynamics
{'q0': [-1.3482526529510004, -0.870886689915784, 2.177141861153415, 0.6771699422262483, 2.0772734922108644, -0.9747077623672942, -1.7078077925338353, 0.0, 0.0016982300253256192, 5.1096778494016315e-05, -2.2602251231881767, -2.34999986832205, 0.5199692664527729, 0.0242897

   1   2.542932e-02  2.51e-04  7.16e-07  1.95e-01       -  1
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
          QP  |   6.59ms (  6.59ms)   6.58ms (  6.58ms)         1
  linesearch  | 153.00us (153.00us) 153.55us (153.55us)         1
      nlp_fg  | 117.00us (117.00us) 117.17us (117.17us)         1
    nlp_grad  | 340.00us (340.00us) 339.89us (339.89us)         1
  nlp_hess_l  |   3.07ms (  3.07ms)   3.08ms (  3.08ms)         1
  nlp_jac_fg  | 543.00us (271.50us) 537.77us (268.88us)         2
       total  |  12.08ms ( 12.08ms)  11.81ms ( 11.81ms)         1
primal residual is : [[0.00025074]]
0
None
Before printing system dynamics
{'q0': [-1.3468216164760276, -0.8699757316582539, 2.1747715431950536, 0.6765143267782864, 2.075011302525254, -0.9736427395059121, -1.7060127510960097, 0.0, 0.0017164255461376554, 9.601172513457318e-05, -2.260423082649726, -2.349999797917166, 0.5199422689934138, 0.023665502388037572, 0.7508481574776127, 0.0015154811963851804, 0.0, 0.00

primal residual is : [[0.000523]]
0
None
Before printing system dynamics
{'q0': [-1.3448673919796106, -0.8687338828594511, 2.1714961058204274, 0.6756525044263952, 2.071883844618848, -0.9721778464209268, -1.7035635641168203, 0.0, 0.0017359287219241492, 0.00016170791179033226, -2.2607125982722858, -2.349999708417759, 0.5199027781537389, 0.022752666506139715, 0.7521128079505167, 0.0025519939535971273, 0.0, 0.0006038843725411532], 'q_dot0': [0.023133793976458816, 0.01467554371341992, -0.038998875674845807, -0.009986021563794356, -0.03724503598182356, 0.01741908945182987, 0.028962202362715718, 0.0, 0.0002095719981753722, 0.0008086843023739909, -0.0035634255277013777, 1.0442673141070058e-06, -0.00048614376472513876, -0.011235045951055804, 0.015567292395393837, 0.012756612303027142, 0.0, 0.0017784089076143465], 's0': array([0.00321508]), 's_dot0': array([0.00899713])}
q
q_dot
s
s_dot
iter      objective    inf_pr    inf_du     ||d||  lg(rg) ls
   0   2.534132e-02  1.25e-03  1.06e-03  0.00e+00

...

   0   2.499916e-02  2.46e-03  7.23e-04  0.00e+00       -  0
   1   2.491145e-02  9.16e-05  9.91e-07  1.78e-01       -  1
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
          QP  |   6.01ms (  6.01ms)   6.01ms (  6.01ms)         1
  linesearch  | 152.00us (152.00us) 151.60us (151.60us)         1
      nlp_fg  | 117.00us (117.00us) 117.81us (117.81us)         1
    nlp_grad  | 384.00us (384.00us) 343.50us (343.50us)         1
  nlp_hess_l  |   1.21ms (  1.21ms)   1.21ms (  1.21ms)         1
  nlp_jac_fg  | 537.00us (268.50us) 537.20us (268.60us)         2
       total  |   9.82ms (  9.82ms)   9.34ms (  9.34ms)         1
primal residual is : [[9.15592944e-05]]
0
None
Before printing system dynamics
{'q0': [-1.3940480964209032, -0.9322207617431275, 2.25578890705152, 0.6452593603213945, 2.1051691098383944, -0.9789776516654821, -1.6637649430817427, 0.0, 0.019296110461408778, 0.07371406716921587, -2.48946863213326, -2.3655533713610217, 0.5015956254200078, -0.6758036783

   1   2.479007e-02  9.29e-05  8.56e-07  1.46e-01       -  1
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
          QP  |   6.04ms (  6.04ms)   6.04ms (  6.04ms)         1
  linesearch  | 148.00us (148.00us) 147.71us (147.71us)         1
      nlp_fg  | 113.00us (113.00us) 114.00us (114.00us)         1
    nlp_grad  | 341.00us (341.00us) 340.99us (340.99us)         1
  nlp_hess_l  |   1.21ms (  1.21ms)   1.21ms (  1.21ms)         1
  nlp_jac_fg  | 518.00us (259.00us) 518.10us (259.05us)         2
       total  |   9.26ms (  9.26ms)   9.00ms (  9.00ms)         1
primal residual is : [[9.28702067e-05]]
0
None
Before printing system dynamics
{'q0': [-1.390645078370567, -0.9232297764181594, 2.248782952943799, 0.6499933511531082, 2.100005435235588, -0.9798567895580694, -1.6674348203487384, 0.0, 0.01962384018741143, 0.07485208498955623, -2.492024161107267, -2.3658788905807646, 0.5015037438409311, -0.6829619917695092, 1.8352887247761818, 0.7630179132824773, 0.0, 0.0230202

Before printing system dynamics
{'q0': [-1.3871390819313953, -0.9140010046860518, 2.241584425450064, 0.6544393483956337, 2.0948636170827455, -0.9810118459384706, -1.6715464749508409, 0.0, 0.019993292134826226, 0.07616596753346391, -2.4949535782042043, -2.3662452755292054, 0.5013960735796391, -0.6910976688740106, 1.849790674436762, 0.7710244207883133, 0.0, 0.02299168582100919], 'q_dot0': [0.04493628393931025, 0.11724754532041028, -0.09192999624006121, 0.05404234316065004, -0.06430644870370486, -0.016089548162127206, -0.05457486056139686, 0.0, 0.004910347464162427, 0.017732482785769332, -0.03935041279863215, -0.004903887443701516, -0.001442459467713042, -0.10885230517216431, 0.19481547743323105, 0.10691851096270526, 0.0, -0.0003391579688339112], 's0': array([0.98705546]), 's_dot0': array([0.02129174])}
q
q_dot
s
s_dot
iter      objective    inf_pr    inf_du     ||d||  lg(rg) ls
   0   2.474665e-02  2.30e-03  4.57e-04  0.00e+00       -  0
   1   2.465506e-02  9.44e-05  7.11e-07  1.12e-01 

   1   2.457045e-02  9.54e-05  2.88e-07  9.69e-02       -  1
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
          QP  |   6.30ms (  6.30ms)   6.30ms (  6.30ms)         1
  linesearch  | 141.00us (141.00us) 141.19us (141.19us)         1
      nlp_fg  | 107.00us (107.00us) 107.64us (107.64us)         1
    nlp_grad  | 322.00us (322.00us) 321.79us (321.79us)         1
  nlp_hess_l  |   1.15ms (  1.15ms)   1.15ms (  1.15ms)         1
  nlp_jac_fg  | 502.00us (251.00us) 502.67us (251.34us)         2
       total  |   9.27ms (  9.27ms)   9.22ms (  9.22ms)         1
primal residual is : [[9.53592195e-05]]
0
None
Before printing system dynamics
{'q0': [-1.384275962685764, -0.906544234569636, 2.235758500991633, 0.6577635834052749, 2.0908263884967635, -0.9821161096676606, -1.6750982102730836, 0.0, 0.020322517710178303, 0.07736348403376368, -2.4976057090303927, -2.3665707953220747, 0.5012964334753558, -0.6984020901858341, 1.8629137213304947, 0.7781859514573538, 0.0, 0.02297

'MPC_SUCCEEDED'

The MPC execution is finished when the termination criteria defined by the monitor is reached.